In [1]:
import ImageLoader
import ImageCompare
import Database

File already exists:
	c:\users\jaden\dropbox\cog\dlibmodels\dlib_models\dlib_face_recognition_resnet_model_v1.dat
File already exists:
	c:\users\jaden\dropbox\cog\dlibmodels\dlib_models\shape_predictor_68_face_landmarks.dat


ModuleNotFoundError: No module named 'Database'

In [1]:
def identify(save=False):
    """
    Takes a picture with configured camera and identifies all of the faces in the picture
    Parameters:
    -----------
    save (boolean):
        whether or not to add the captured image to the database
    
    Returns:
    --------
    names (list)
        the list of the name of each person in the picture
    """
    descs = ImageLoader.describe()
    names = ImageCompare.compare_faces(descs, Database.database)
    if save:
        Database.add_iamge(desc)
    return names

In [2]:
def display_labeled():
    """
    Displays an image with boxes around the faces and their labels 
    Parameters:
    -----------
    
    Returns:
    --------
    
    """